In [1]:
import numpy as np
n_features = 'all'
term_pred = 'short_term'
data_file_name = 'data_y10'
training_id_file_name = 'all_training_ID_outerloop_cohort_0_10'
endpt = 10;
eval_times = 365.25*np.r_[1, np.arange(2, endpt+1, 2)]
l2_final=0.1
work_dir = '/home/idies/workspace/Storage/hnguye78/persistent/CARDIA_project/cvd_outcome_rerun2'


In [2]:
# %pip install tensorflow
# %pip install lifelines

#!y | pip uninstall statsmodels 
# %pip install statsmodels==0.11.0

# %pip install keras

In [3]:

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import matplotlib
import math
from lifelines import KaplanMeierFitter
from lifelines import CoxPHFitter
from lifelines.utils import concordance_index
from sklearn.preprocessing import StandardScaler
from scipy import stats
import time


In [4]:
snippets_dir = '/home/idies/workspace/Storage/hnguye78/persistent/CARDIA_project/cvd_outcome_rerun'+ '/code/snippet'
import os, sys
sys.path.append(os.path.abspath(snippets_dir))
import cox_nnet
from cox import *


In [5]:
load_dir = work_dir+ '/csv_files'

# load data:
data_full = pd.read_csv(load_dir+'/'+data_file_name+'.csv')
data_full = data_full.select_dtypes(include =[np.number])



In [6]:
data_full

,ID,event,time,sex,race,cholesterol,hdl,hbp.medication,smoker,diabetes,...,C08MELAG_YEAR,C08SKNAG_YEAR,C08BRNAG_YEAR,C08STMAG_YEAR,C08OCAAG_YEAR,C08GALDA_YEAR,C08PRGAG_YEAR,C08KYSAG_YEAR,C08URNAG_YEAR,count_na
0,100016012504,0,3689.025,0,0,186.0,38.0,0,1,0,...,0,0,0,0,0,0,0,0,0,0.048673
1,100023004268,0,3689.025,0,0,166.0,43.0,0,0,0,...,0,0,0,0,0,0,0,0,0,0.055310
2,100033323702,0,3689.025,0,0,234.0,52.0,0,0,0,...,0,0,0,0,0,0,0,0,0,0.077434
3,100048415270,0,1169.000,0,1,137.0,40.0,0,1,0,...,0,0,0,0,0,0,0,0,18,0.008850
4,100056526386,0,3689.025,0,1,199.0,44.0,0,0,0,...,0,0,0,0,0,0,0,0,0,0.008850
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4309,416761219907,0,3689.025,1,0,225.0,57.0,0,0,0,...,0,0,0,0,0,0,14,0,0,0.048673
4310,416771521620,0,3689.025,0,1,204.0,76.0,0,0,0,...,0,0,0,0,0,0,0,0,0,0.004425
4311,416783315386,0,3689.025,1,0,159.0,58.0,0,0,0,...,0,0,0,0,0,0,8,0,0,0.112832
4312,416796224310,0,3689.025,1,1,164.0,55.0,0,0,0,...,0,0,0,0,0,0,14,0,0,0.070796


In [7]:
# load training id:
loaddir = work_dir+ '/csv_files'
trainingid_all = pd.read_csv(loaddir+'/'+training_id_file_name+'.csv')

## standardize feature space, then merge back to label space:
feature_space = data_full.drop(['ID','event','time'], axis = 1)
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler().fit(feature_space)
scaled_feature_space = scaler.transform(feature_space)
scaled_feature_space_df = pd.DataFrame(data=scaled_feature_space[0:,0:])
scaled_feature_space_df.insert(0, 'ID', data_full['ID'], True)
label = data_full.loc[:,['ID','event','time']]
data_full = pd.merge(label, scaled_feature_space_df)

In [8]:
data_full

,ID,event,time,0,1,2,3,4,5,6,...,444,445,446,447,448,449,450,451,452,453
0,100016012504,0,3689.025,-1.106215,-0.970306,0.235193,-1.090061,-0.126551,1.580754,-0.150863,...,-0.022243,-0.024272,-0.021537,-0.015227,-0.056678,-0.087086,-0.626333,-0.090552,-0.124130,0.773715
1,100023004268,0,3689.025,-1.106215,-0.970306,-0.356134,-0.732871,-0.126551,-0.632609,-0.150863,...,-0.022243,-0.024272,-0.021537,-0.015227,-0.056678,-0.087086,-0.626333,-0.090552,-0.124130,0.979473
2,100033323702,0,3689.025,-1.106215,-0.970306,1.654379,-0.089929,-0.126551,-0.632609,-0.150863,...,-0.022243,-0.024272,-0.021537,-0.015227,-0.056678,-0.087086,-0.626333,-0.090552,-0.124130,1.665334
3,100048415270,0,1169.000,-1.106215,1.030602,-1.213559,-0.947185,-0.126551,1.580754,-0.150863,...,-0.022243,-0.024272,-0.021537,-0.015227,-0.056678,-0.087086,-0.626333,-0.090552,8.591018,-0.460833
4,100056526386,0,3689.025,-1.106215,1.030602,0.619556,-0.661433,-0.126551,-0.632609,-0.150863,...,-0.022243,-0.024272,-0.021537,-0.015227,-0.056678,-0.087086,-0.626333,-0.090552,-0.124130,-0.460833
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4309,416761219907,0,3689.025,0.903984,-0.970306,1.388282,0.267261,-0.126551,-0.632609,-0.150863,...,-0.022243,-0.024272,-0.021537,-0.015227,-0.056678,-0.087086,1.968180,-0.090552,-0.124130,0.773715
4310,416771521620,0,3689.025,-1.106215,1.030602,0.767388,1.624583,-0.126551,-0.632609,-0.150863,...,-0.022243,-0.024272,-0.021537,-0.015227,-0.056678,-0.087086,-0.626333,-0.090552,-0.124130,-0.598005
4311,416783315386,0,3689.025,0.903984,-0.970306,-0.563099,0.338699,-0.126551,-0.632609,-0.150863,...,-0.022243,-0.024272,-0.021537,-0.015227,-0.056678,-0.087086,0.856246,-0.090552,-0.124130,2.762710
4312,416796224310,0,3689.025,0.903984,1.030602,-0.415267,0.124385,-0.126551,-0.632609,-0.150863,...,-0.022243,-0.024272,-0.021537,-0.015227,-0.056678,-0.087086,1.968180,-0.090552,-0.124130,1.459575


In [9]:
# Outerloop:
for fold in range(25):

    #fold = 24

    print('')
    print('FOLD '+str(fold)+':')
    print('')

    trainingid = trainingid_all.iloc[:,fold]
    trainingid = trainingid[~np.isnan(trainingid)]
    eligible_id = data_full['ID'][data_full['ID'].isin(trainingid)]
    train_df = data_full.loc[data_full['ID'].isin(eligible_id),:]
    del train_df['ID']
    test_df = data_full.loc[~data_full['ID'].isin(eligible_id),:]
    del test_df['ID']


    model_params = dict(node_map = None, input_split = None)
    search_params = dict(method = "nesterov"
                         , learning_rate=0.0001
                         , momentum=0.9
                         , max_iter=200000
                         , stop_threshold=0.995
                         , patience=1000
                         , patience_incr=2
                         , rand_seed = 9114
                         , eval_step=23
                         , lr_decay = 0.9
                         , lr_growth = 1.0)

    # #train final model
    L2_reg = 0.1
    featurespace_train = train_df.drop(['time','event'],axis =1).values
    featurespace_test = test_df.drop(['time','event'],axis =1).values
    model_params = dict(node_map = None, input_split = None, L2_reg=0.1)
    cox_nnet_model, cox_nnet_cost_iter = cox_nnet.trainCoxMlp(featurespace_train
                                                              , train_df.time.values
                                                              , train_df.event.values
                                                              , model_params
                                                              , search_params
                                                              , verbose=True)
    cox_nnet_theta_train = cox_nnet_model.predictNewData(featurespace_train)
    cox_nnet_theta_test = cox_nnet_model.predictNewData(featurespace_test)

    #discrimination on train, test sets
    c_train = concordance_index(train_df.time,-cox_nnet_theta_train, train_df.event)
    print('C on training set:' + str(c_train))
    c_test = concordance_index(test_df.time,-cox_nnet_theta_test, test_df.event)
    print('C on test set:' + str(c_test))

    predicted_risk = cox_nnet_theta_test
    predicted_risk = pd.DataFrame(data=predicted_risk)
    times, H0 = cox_basehaz(cox_nnet_theta_train.astype('float32').flatten(), train_df.time.values, train_df.event.values.astype(bool))
    trained_time = pd.DataFrame(data = times)
    pred_prob_surv = cox_pred_surv(cox_nnet_theta_test.astype('float32').flatten(), H0)
    pred_prob_surv = pd.DataFrame(data=pred_prob_surv)


    savedir = os.path.join(work_dir, 'csv_files/cox_nnet/'+str(n_features)+'_features/'+term_pred)
    try: 
        os.makedirs(savedir)
    except OSError:
        if not os.path.isdir(savedir):
            raise
    actual_fold = fold+1

    predicted_risk.to_csv(savedir+'/lp_fold_'+str(actual_fold)+'.csv', index = None, header = True)
    pred_prob_surv.to_csv(savedir+'/pred_prob_surv_fold_'+str(actual_fold)+'.csv', index = None, header = True)
    trained_time.to_csv(savedir+'/trained_time_fold_'+str(actual_fold)+'.csv', index = None, header = True)




FOLD 7:

Using nesterov accelerated gradient
training model
cost: 4.243379, iteration: 0
cost: 4.216204, iteration: 23
cost: 4.179228, iteration: 46
cost: 4.141700, iteration: 69
cost: 4.104437, iteration: 92
cost: 4.067509, iteration: 115
cost: 4.030919, iteration: 138
cost: 3.994664, iteration: 161
cost: 3.958743, iteration: 184
cost: 3.923150, iteration: 207
cost: 3.887885, iteration: 230
cost: 3.852943, iteration: 253
cost: 3.818321, iteration: 276
cost: 3.784017, iteration: 299
cost: 3.750028, iteration: 322
cost: 3.716350, iteration: 345
cost: 3.682982, iteration: 368
cost: 3.649919, iteration: 391
cost: 3.617160, iteration: 414
cost: 3.584702, iteration: 437
cost: 3.552541, iteration: 460
cost: 3.520675, iteration: 483
cost: 3.489102, iteration: 506
cost: 3.457818, iteration: 529
cost: 3.426821, iteration: 552
cost: 3.396109, iteration: 575
cost: 3.365678, iteration: 598
cost: 3.335526, iteration: 621
cost: 3.305652, iteration: 644
cost: 3.276051, iteration: 667
cost: 3.246721,